In [ ]:
import os
import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xa
from matplotlib.colors import LogNorm

In [ ]:
buoys = pd.read_feather('data/segments.feather').dropna()
idir = '../NERSC_arctic25km_sea_ice_age_v2p1/nc'

In [ ]:
year = 2020
sia_files = sorted(glob.glob(f'{idir}/{year}/arctic25km_sea_ice_age_v2p1_*.nc'))
print(len(sia_files))

with xa.open_dataset(sia_files[0]) as ds:
    ds_x = ds.x.values
    ds_y = ds.y.values
    ds_status_flag = ds.status_flag.values
land = np.where(ds_status_flag[0] == 2, 1, np.nan)

In [ ]:
for year in range(2024, 1990, -1):
    ofile = f'data/buoys_sia_{year}.feather'
    if os.path.exists(ofile):
        print(f'{ofile} exists, skipping')
        continue

    print(f'Processing {year}')
    sia_files = sorted(glob.glob(f'{idir}/{year}/arctic25km_sea_ice_age_v2p1*.nc'))[::2]
    print(len(sia_files))

    with xa.open_dataset(sia_files[0]) as ds:
        ds_x = ds.x.values
        ds_y = ds.y.values

    concs = []
    with xa.open_mfdataset(sia_files, combine='by_coords') as ds:
        ds_time = ds.time.values
        for var_name in sorted(ds.variables, reverse=True):
            if 'conc_' in var_name and 'uncert' not in var_name:
                concs.append(ds.variables[var_name].values)
        ds_sia = ds.sea_ice_age.values
    concs = np.array(concs)
    conc_cumsum = concs.cumsum(axis=0)[::-1]

    buoys_sub = buoys[(buoys.index.year == year)].copy()
    print(buoys_sub.shape)

    time_indices = np.array([np.argmin(np.abs(ds_time - np.datetime64(date))).item() for date in buoys_sub.index])
    cols = np.round((buoys_sub.x - ds_x[0]) / (ds_x[-1] - ds_x[0]) * ds_x.size).astype(int)
    rows = np.round((buoys_sub.y - ds_y[0]) / (ds_y[-1] - ds_y[0]) * ds_y.size).astype(int)
    rows = np.clip(rows, 0, ds_y.size - 1)
    cols = np.clip(cols, 0, ds_x.size - 1)
    buoy_sia = ds_sia[time_indices, rows, cols]
    buoy_conc_cumsum = conc_cumsum[:, time_indices, rows, cols]

    buoys_sub['sia'] = buoy_sia
    for i in range(buoy_conc_cumsum.shape[0]):
        buoys_sub[f'conc{i}'] = buoy_conc_cumsum[i]
    buoys_sub.dropna().to_feather(ofile)